In [ ]:
import tensorflow as tf
print(tf.__version__)

# Validation, regularisation and callbacks

***
<a id="coding_tutorial_1"></a>
## Validation sets

#### Load the data

In [ ]:
# Load the diabetes dataset
from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()
print (diabetes_dataset["DESCR"])


In [ ]:
# Save the input and target variables
#print(diabetes_dataset.keys())

data = diabetes_dataset["data"]
targets = diabetes_dataset["target"]

In [ ]:
# Normalise the target data (this will make clearer training curves)

#targets.mean(axis=0)
targets = ( targets - targets.mean(axis=0) )/ targets.std()

targets.mean()

In [ ]:
# Split the data into train and test sets

from sklearn.model_selection import train_test_split

train_data, test_data ,train_targets, test_targets = train_test_split(data, targets, test_size=0.1)

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(train_targets.shape)
print(test_targets.shape)

#### Train a feedforward neural network model

In [ ]:
# Build the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten#, Conv2D, MaxPooling2D
help(Dense)

In [ ]:
def get_model():
    model=Sequential([
        Dense (128, activation='relu',input_shape=(train_data.shape[1],), name="dense1"),
        Dense (128, activation='relu', name="dense2"),
        Dense (128, activation='relu', name="dense3"),
        Dense (128, activation='relu', name="dense4"),
        Dense (128, activation='relu', name="dense5"),
        Dense (128, activation='relu', name="dense6"),
        Dense (1, activation='relu', name="dense7")
    ])
    return model

In [ ]:
# Print the model summary
model = get_model()
print(model.summary())


In [ ]:
# Compile the model

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [ ]:
# Train the model, with some of the data reserved for validation
history = model.fit(train_data, train_targets, epochs=100, 
          validation_split=0.15, 
          batch_size=64, 
          verbose=2) #verbose=False


In [ ]:
# Evaluate the model on the test set
model.evaluate(test_data , test_targets, verbose=0 )
#dir(model)


#### Plot the learning curves

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot the training and validation loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

***
<a id="coding_tutorial_2"></a>
## Model regularisation

#### Adding regularisation with weight decay and dropout

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [ ]:
def get_regularised_model(wd, rate):
    model = Sequential([
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd), input_shape=(train_data.shape[1],)),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd)),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd)),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd)),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd)),
        Dropout(rate),
        Dense(128, activation="relu"),
        Dropout(rate),
        Dense(1)
    ])
    return model

In [ ]:
# Re-build the model with weight decay and dropout layers

rmodel = get_regularised_model(0.00001, 0.3)

In [ ]:
# Compile the model

rmodel.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [ ]:
# Train the model, with some of the data reserved for validation
history = rmodel.fit(train_data, train_targets, epochs=100, 
          validation_split=0.15, 
          batch_size=64, 
          verbose=2)


In [ ]:
# Evaluate the model on the test set
rmodel.evaluate(test_data , test_targets, verbose=0)
#history

#### Plot the learning curves

In [ ]:
# Plot the training and validation loss

import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

***
<a id="coding_tutorial_3"></a>
## Introduction to callbacks

#### Example training callback

In [ ]:
# Write a custom callback
from tensorflow.keras.callbacks import Callback

class TrainingCallback(Callback):
    def on_train_begin(self, logs=None):
        print ("Starting training...")
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch}")
    def on_train_batch_begin(self, batch, logs=None):
        print(f"Training: Starting batch {batch}")
    def on_train_batch_end(self, batch, logs=None):
        print(f"Training: Finished batch {batch}")
    def on_epoch_end(self, epoch, logs=None):
        print(f"Training: Fininshed epoch {epoch}")    
    def on_train_end(self, logs=None):
        print ("Finished training.")

class TestingCallback(Callback):
    def on_test_begin(self, logs=None):
        print ("Starting testing...")
    def on_test_batch_begin(self, batch, logs=None):
        print(f"Testing: Starting batch {batch}")
    def on_test_batch_end(self, batch, logs=None):
        print(f"Testing: Finished batch {batch}")
    def on_test_end(self, logs=None):
        print ("Finished testing.")

class PredcitionCallback(Callback):
    def on_predict_begin(self, logs=None):
        print ("Starting predict ing...")
    def on_predict_batch_begin(self, batch, logs=None):
        print(f"Predicting: Starting batch {batch}")
    def on_predict_bach_end(self, batch, logs=None):
        print(f"Predicting: Finished batch {batch}")
    def on_predict_end(self, logs=None):
        print ("Finished predicting.")

In [ ]:
# Re-build the model

rmodel2 = get_regularised_model(0.00001, 0.3)

In [ ]:
# Compile the model
rmodel2.compile(optimizer="adam", loss="mae", metrics=["mae"])


#### Train the model with the callback

In [ ]:
# Train the model, with some of the data reserved for validation
history = rmodel2.fit(train_data, train_targets, epochs=5, 
                      validation_split=0.15, 
                      batch_size=128, 
                      verbose=0,
                      callbacks=[TrainingCallback()])


In [ ]:
# Evaluate the model
rmodel2.evaluate(test_data , test_targets, verbose=0, callbacks=[TestingCallback()])


In [ ]:
# Make predictions with the model

rmodel2.predict(test_data, verbose=0, callbacks=[PredcitionCallback()])

***
<a id="coding_tutorial_4"></a>
## Early stopping / patience

#### Re-train the models with early stopping

In [ ]:
# Re-train the unregularised model
unregularized_model = get_model()
unregularized_model.compile(optimizer="adam", loss="mse")
unreg_history = unregularized_model.fit(
    train_data, train_targets, 
    epochs=100, validation_split=0.15, batch_size=64,
    verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping()])

In [ ]:
# Evaluate the model on the test set
unregularized_model.evaluate(test_data, test_targets, verbose=2)


In [ ]:
# Re-train the regularised model
regularized_model = get_regularised_model(1e-8, 0.2)
regularized_model.compile(optimizer="adam", loss="mse")
reg_history = regularized_model.fit(
    train_data, train_targets, 
    epochs=100, validation_split=0.15, batch_size=64,
    verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping()])


In [ ]:
# Evaluate the model on the test set
regularized_model.evaluate(
    test_data, test_targets, verbose=2)


#### Plot the learning curves

In [ ]:
# Plot the training and validation loss

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(unreg_history.history['loss'])
plt.plot(unreg_history.history['val_loss'])
plt.title('Unregularised model: loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(reg_history.history['loss'])
plt.plot(reg_history.history['val_loss'])
plt.title('Regularised model: loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

plt.show()

In [ ]:
# Re-train the unregularised model with patience
unregularized_model = get_model()
unregularized_model.compile(optimizer="adam", loss="mse")
unreg_history_patience = unregularized_model.fit(
    train_data, train_targets, 
    epochs=100, validation_split=0.15, batch_size=64,
    verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
# Re-train the regularised model with patience
regularized_model = get_regularised_model(1e-8, 0.2)
regularized_model.compile(optimizer="adam", loss="mse")
reg_history_patience = regularized_model.fit(
    train_data, train_targets, 
    epochs=100, validation_split=0.15, batch_size=64,
    verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

In [ ]:

fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(unreg_history_patience.history['loss'])
plt.plot(unreg_history_patience.history['val_loss'])
plt.title('Unregularised model: loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(reg_history_patience.history['loss'])
plt.plot(reg_history_patience.history['val_loss'])
plt.title('Regularised model: loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

plt.show()